# F.R.I.E.N.D.S.

In this project we will analyze the sentiment of every line in the TV show Friends. For this we will first download the TV Show script from the Kaggle dataset by user divyansh22 using the kaggle API.

In [46]:
# If you havent already ran the appropriate script, uncomment the follwing line and run this cell of this jupyter notebook.
#%run 1-download_and_clean_data.py

100%|██████████| 1.67M/1.67M [00:00<00:00, 11.9MB/s]


Cleaning Friends script...
Extracting scene info...
Fetching season data...
Matching episode names with seasons...
Matching episode names with script...
Processing character names...
Starting sentiment analyisis with SIA... this will take a VERY long time
Done! 
Starting sentiment analyisis with TextBlob... this one is faster
Done!
Renaming columns for SQL...
Done! exported to 'data/' 


This script downloads and processes the .txt file of the script and generates 3 separate dataframes that are then exported as CSV files. These data frames are:

In [47]:
friends_script.head()

,f_char,f_line,scene_number,sia_neg,sia_neu,sia_pos,sia_compound,tb_polarity,tb_subjectivity
0,Monica,There's nothing to tell! He's just some guy I ...,0,0.000,1.000,0.000,0.0000,0.000000,0.000000
1,Joey,"C'mon, you're going out with the guy! There's ...",0,0.221,0.779,0.000,-0.5696,-0.625000,0.900000
2,Chandler,"All right Joey, be nice. So does he have a hum...",0,0.000,0.792,0.208,0.4871,0.442857,0.767857
3,Phoebe,"Wait, does he eat chalk?",0,0.000,1.000,0.000,0.0000,0.000000,0.000000
4,Phoebe,"Just, 'cause, I don't want her to go through w...",0,0.104,0.896,0.000,-0.1316,0.000000,0.000000


In [48]:
friends_script.shape

(67507, 9)

friends_script DF that has 67.507 lines. This DF has a single row for every line spoken in the show. It includes the scene number, character, text and sentiment analysis for the line.

In [49]:
f_scene_info.head()

,scene_number,episode,scene
0,0,1,"Central Perk, Chandler, Joey, Phoebe, and Moni..."
1,1,1,"The Subway, Phoebe is singing for change"
2,2,1,"Ross's Apartment, the guys are there assemblin..."
3,3,1,"A Restaurant, Monica and Paul are eating"
4,4,1,Ross's Apartment; Ross is pacing while Joey an...


In [50]:
f_scene_info.shape

(2971, 3)

f_scene_info DF that has 2.971 rows. This DF has a single line for every scene in the show and it has columns for the scene number, episode in which it is in and the description of the scene.

In [51]:
f_seasons.head()

,ep_number_overall,ep_number_season,ep_title,directed_by,written_by,org_air_date,prod_code,us_viewers_mm,season,rating_1,rating_2,special_num,us_viewers_mm_2
0,1,1,The One Where Monica Gets A New Roommate,James Burrows,David Crane & Marta Kauffman,"September 22, 1994",475085.0,21.5,1,NaN,NaN,NaN,NaN
1,2,2,The One With The Sonogram At The End,James Burrows,David Crane & Marta Kauffman,"September 29, 1994",456652.0,20.2,1,NaN,NaN,NaN,NaN
2,3,3,The One With The Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,"October 6, 1994",456651.0,19.5,1,NaN,NaN,NaN,NaN
3,4,4,The One With George Stephanopoulos,James Burrows,Alexa Junge,"October 13, 1994",456654.0,19.7,1,NaN,NaN,NaN,NaN
4,5,5,The One With The East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,"October 20, 1994",456653.0,18.6,1,NaN,NaN,NaN,NaN


In [52]:
f_seasons.shape

(230, 13)

f_seasons DF which has 230 rows. It has a single row for every episode in the show along with all the information available for that episode taken from [wikipedia](https://en.wikipedia.org/wiki/List_of_Friends_episodes).

After this process was complete we then continued and uploaded this inforamtion to our local server for processing in sql.

In [53]:
# If you havent already ran the appropriate script, uncomment the following line and run this cell of this jupyter notebook.
#%run 2-uploading_to_sql.py

In [54]:
#Average sentiment (sia_compound) of all lines of top 8 characters (by line count) of every season
sent_by_char_by_season = pd.read_sql_query("""

    SELECT t.season AS season, t.f_char AS charact, t.num_lines, t.sentimient AS sentiment
        FROM (
                SELECT sea.season AS season, scr.f_char AS f_char, COUNT(scr.f_char) AS num_lines, AVG(scr.sia_compound) AS sentimient,
                    ROW_NUMBER() OVER (PARTITION BY sea.season ORDER BY COUNT(scr.f_char) DESC) AS row_num
                FROM script AS scr
                LEFT JOIN scenes AS sce
                    ON scr.scene_number = sce.scene_number
                LEFT JOIN seasons AS sea
                    ON sea.ep_number_overall = sce.episode
                GROUP BY sea.season, scr.f_char
            ) t
    WHERE t.row_num <= 8
    ORDER BY t.season ASC, sentiment DESC;

""", engine)
sent_by_char_by_season

,season,charact,num_lines,sentiment
0,1,Rachel,1127,0.140176
1,1,All,134,0.127669
2,1,Chandler,1151,0.127106
3,1,Phoebe,853,0.114670
4,1,Monica,1213,0.113511
...,...,...,...,...
67,10,Mike,365,0.135254
68,10,Ross,1853,0.134299
69,10,Monica,1686,0.122826
70,10,Joey,1747,0.111933


In [55]:
# Average sentiment (sia_compound) of all lines of every season
sent_by_season = pd.read_sql_query("""

        SELECT sea.season AS season, AVG(scr.sia_compound) AS sentiment
        FROM script AS scr
        LEFT JOIN scenes AS sce
            ON scr.scene_number = sce.scene_number
        LEFT JOIN seasons AS sea
            ON sea.ep_number_overall = sce.episode

        GROUP BY season
        ORDER BY sentiment DESC
        ;

""", engine)
sent_by_season

,season,sentiment
0,6,0.135811
1,10,0.133944
2,5,0.133375
3,7,0.131802
4,9,0.125639
5,4,0.123779
6,3,0.118617
7,1,0.111933
8,2,0.105358


In [56]:
# Average sentiment (sia_compound) of all lines of main characters:
sent_by_character = pd.read_sql_query("""

        SELECT scr.f_char AS charact, count(scr.f_char) AS num_lines, AVG(scr.sia_compound) AS sentiment
        FROM script AS scr
        LEFT JOIN scenes AS sce
            ON scr.scene_number = sce.scene_number
        LEFT JOIN seasons AS sea
            ON sea.ep_number_overall = sce.episode
        GROUP BY charact
        HAVING charact IN ('Rachel', 'Monica','Phoebe','Ross','Joey','Chandler')
        ORDER BY sentiment DESC
        ;

""", engine)
sent_by_character

,charact,num_lines,sentiment
0,Rachel,9957,0.143365
1,Phoebe,8234,0.139062
2,Joey,9110,0.125653
3,Chandler,9386,0.121130
4,Ross,10090,0.121002
5,Monica,9175,0.116530


In [57]:
# Number of lines of main characters per character per season:
lines_per_charac_per_season = pd.read_sql_query("""

        SELECT sea.season AS season, scr.f_char AS charact, count(scr.f_char) AS num_lines
        FROM script AS scr
        LEFT JOIN scenes AS sce
            ON scr.scene_number = sce.scene_number
        LEFT JOIN seasons AS sea
            ON sea.ep_number_overall = sce.episode
            
        GROUP BY season, charact
        HAVING charact IN ('Rachel', 'Monica','Phoebe','Ross','Joey','Chandler')
        ORDER BY season, num_lines DESC
        ;

""", engine)
lines_per_charac_per_season

,season,charact,num_lines
0,1,Ross,1308
1,1,Monica,1213
2,1,Chandler,1151
3,1,Rachel,1127
4,1,Joey,891
5,1,Phoebe,853
6,2,Ross,399
7,2,Monica,360
8,2,Chandler,342
9,2,Phoebe,287


Using this data we can use Tableau to graph it and gain insights as to sentiment by character, season, episode, etc.

In [58]:
from IPython.display import IFrame
IFrame(src="<div class='tableauPlaceholder' id='viz1677583594807' style='position: relative'><noscript><a href='#'><img alt='F.R.I.E.N.D.S. Sentiment analysis ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Fr&#47;FriendsSentimentAnalysis&#47;F_R_I_E_N_D_S_Sentimentanalysis&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='FriendsSentimentAnalysis&#47;F_R_I_E_N_D_S_Sentimentanalysis' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Fr&#47;FriendsSentimentAnalysis&#47;F_R_I_E_N_D_S_Sentimentanalysis&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1677583594807');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1016px';vizElement.style.height='991px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>", width=800, height=600)